# Train images

In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

from exp.nb_Tesis import *
from PIL import ImageOps

In [2]:

from torchvision.utils import save_image


In [29]:
path_train_imgs_2017 = Path('2017/data/ISIC-2017_Training_Data/')
path_valid_imgs_2017 = Path('2017/data/ISIC-2017_Validation_Data/')

In [30]:
list_of_train_imgs_paths = get_image_files(path_train_imgs_2017)
list_of_valid_imgs_paths = get_image_files(path_valid_imgs_2017)

In [49]:
import warnings
warnings.filterwarnings("ignore", category=UserWarning, module="torch.nn.functional")

# Train 2019

In [3]:
min_dim = 352
path_train_2019 = Path('./ISIC_2019_Training_Input')
path_train_2019_small = path_train_2019.parent / f'small_{min_dim}_ISIC_2019_Training_Input'

path_test_2018 = Path('./ISIC2018_Task3_Test_Input')
path_test_2018_small = path_test_2018.parent / f'small_{min_dim}_ISIC2018_Task3_Test_Input'

crop = True

In [4]:
!ls small_64_ISIC_2019_Training_Input/ | wc -l

25332


In [12]:
!file ./$path_test_2018/ISIC_0034526.jpg

./ISIC2018_Task3_Test_Input/ISIC_0034526.jpg: JPEG image data, baseline, precision 8, 600x450, frames 3


In [5]:
from fastai.vision import *

# path_hr = path_train_2019
# path_mr = path_train_2019_small

path_hr = path_test_2018
path_mr = path_test_2018_small

il = ImageList.from_folder(path_hr)

def resize_one(fn, i, path, size):
    dest = path/fn.relative_to(path_hr)
    dest.parent.mkdir(parents=True, exist_ok=True)
    img = PIL.Image.open(fn)
    targ_sz = resize_to(img, size, use_min=True)
    img = img.resize(targ_sz, resample=PIL.Image.BILINEAR).convert('RGB')
    img.save(dest, quality=100)
    if crop:
        img = open_image(dest)
        crop(img,min_dim)
        img.save(dest)

# create smaller image sets the first time this nb is run
sets = [(path_mr, min_dim)]
for p,size in sets:
    if not p.exists(): 
        print(f"resizing to {size} into {p}")
        parallel(partial(resize_one, path=p, size=size), il.items)

# Test 2017

In [5]:
path_train_2019 = Path('/data2/mike/ISIC-2017_Test_v2_Data/')

from fastai.vision import *

path_hr = path_train_2019
path_mr = path_train_2019/'small-224'

il = ImageList.from_folder(path_hr)

def resize_one(fn, i, path, size):
    dest = path/fn.relative_to(path_hr)
    dest.parent.mkdir(parents=True, exist_ok=True)
    img = PIL.Image.open(fn)
    targ_sz = resize_to(img, size, use_min=True)
    img = img.resize(targ_sz, resample=PIL.Image.BILINEAR).convert('RGB')
    img.save(dest, quality=100)

# create smaller image sets the first time this nb is run
sets = [(path_mr, 224)]
for p,size in sets:
    if not p.exists(): 
        print(f"resizing to {size} into {p}")
        parallel(partial(resize_one, path=p, size=size), il.items)